In [ ]:
from yugiquery import *
header('Cards')

---

Table of Contents
=================

*   [1  Data aquisition](#Data-aquisition)
    *   [1.1  Fetch online data](#Fetch-online-data)
    *   [1.2  Merge data](#Merge-data)
    *   [1.3  Save data](#Save-data)
*   [3  Check changes](#Check-changes)
    *   [3.1  Load previous data](#Load-previous-data)
    *   [3.2  Generate changelog](#Generate-changelog)
*   [4  Data visualization](#Data-visualization)
    *   [4.1  Full data](#Full-data)
    *   [4.2  Card types](#Card-types)
    *   [4.3  Monsters](#Monsters)
        *   [4.3.1  Attributes](#Attributes)
        *   [4.3.2  Primary types](#Primary-types)
            *   [4.3.2.1  Has effect discrimination](#Has-effect-discrimination)
            *   [4.3.2.2  Is pendulum discrimination](#Is-pendulum-discrimination)
            *   [4.3.2.3  By attribute](#By-attribute)
        *   [4.3.3  Secondary types](#Secondary-types)
            *   [4.3.3.1  By attribute](#By-attribute)
            *   [4.3.3.2  By secondary type](#By-secondary-type)
        *   [4.3.4  Monster types](#Monster-types)
            *   [4.3.4.1  By Attribute](#By-Attribute)
            *   [4.3.4.2  By primary type](#By-primary-type)
            *   [4.3.4.3  By secondary type](#By-secondary-type)
        *   [4.3.5  Effect type](#Effect-type)
        *   [4.3.6  ATK](#ATK)
        *   [4.3.7  DEF](#DEF)
        *   [4.3.8  Level/Rank](#Level/Rank)
            *   [4.3.8.1  ATK statistics](#ATK-statistics)
            *   [4.3.8.2  DEF statistics](#DEF-statistics)
        *   [4.3.9  Pendulum scale](#Pendulum-scale)
            *   [4.3.9.1  ATK statistics](#ATK-statistics)
            *   [4.3.9.2  DEF statistics](#DEF-statistics)
            *   [4.3.9.3  Level/Rank statistics](#Level/Rank-statistics)
        *   [4.3.10  Link](#Link)
            *   [4.3.10.1  ATK statistics](#ATK-statistics)
        *   [4.3.11  Link Arrows](#Link-Arrows)
            *   [4.3.11.1  By combination](#By-combination)
            *   [4.3.11.2  By unique](#By-unique)
            *   [4.3.11.3  By link](#By-link)
    *   [4.4  Spell & Trap](#Spell-&-Trap)
        *   [4.4.1  Properties](#Properties)
        *   [4.4.2  Effect type](#Effect-type)
            *   [4.4.2.1  Spell & Trap discrimination](#Spell-&-Trap-discrimination)
    *   [4.5  Archseries](#Archseries)
        *   [4.5.1  By card type](#By-card-type)
        *   [4.5.2  By primary type](#By-primary-type)
        *   [4.5.3  By secondary type](#By-secondary-type)
        *   [4.5.4  By monster type](#By-monster-type)
        *   [4.5.5  By property](#By-property)
    *   [4.6  Artworks](#Artworks)
        *   [4.6.1  By card type](#By-card-type)
        *   [4.6.2  By primary type](#By-primary-type)
    *   [4.7  Errata](#Errata)
        *   [4.7.1  By card type](#By-card-type)
        *   [4.7.2  By primary type](#By-primary-type)
        *   [4.7.3  By artwork](#By-artwork)
    *   [4.8  TCG & OCG status](#TCG-&-OCG-status)
        *   [4.8.1  TGC status](#TGC-status)
            *   [4.8.1.1  By card type](#By-card-type)
            *   [4.8.1.2  By monster type](#By-monster-type)
            *   [4.8.1.3  By archseries](#By-archseries)
        *   [4.8.2  OCG status](#OCG-status)
            *   [4.8.2.1  By card type](#By-card-type)
            *   [4.8.2.2  By monster type](#By-monster-type)
            *   [4.8.2.3  By archseries](#By-archseries)
        *   [4.8.3  TCG vs. OCG status](#TCG-vs.-OCG-status)
*   [5  Extras](#Extras)
    *   [5.1 Multiple secondary types](#Multiple-secondary-types)
    *   [5.2 Not yet released](#Not-yet-released)
    *   [5.3 Counters and Tokens](#Counters-and-Tokens)
*   [6  Epilogue](#Epilogue)
    *   [6.1  HTML export](#HTML-export)
<!-- *   [6.2  Git](#Git) -->

# Data aquisition

## Fetch online data

In [ ]:
# Timestamp
timestamp = pd.Timestamp.now()

In [ ]:
# Fetch Monster
monster_df = fetch_monster()

In [ ]:
# Fetch Spell
spell_df = fetch_st(st='Spell')

# Fetch Trap
trap_df = fetch_st(st='Trap')

In [ ]:
# Fetch Counter
counter_df = fetch_counter()

# Fetch errata
errata_df = fetch_errata()

## Merge data

In [ ]:
# Merge errata
monster_with_token_df = merge_errata(monster_df, errata_df, drop=True)
spell_df = merge_errata(spell_df, errata_df, drop=True)
trap_df = merge_errata(trap_df, errata_df, drop=True)
counter_df = merge_errata(counter_df, errata_df, drop=True)

# Separate tokens from monsters
token_df = monster_with_token_df[monster_with_token_df['Primary type']=='Monster Token']
monster_df = monster_with_token_df[monster_with_token_df['Primary type']!='Monster Token']

# Add tokens to counters data frame
counter_df = pd.concat([counter_df, token_df], ignore_index=True, axis=0).sort_values('Name').reset_index(drop=True)

# Create combined spell and trap data frame
st_df = pd.concat([spell_df, trap_df], ignore_index=True, axis=0).sort_values('Name').reset_index(drop=True)

# Create combined monster, spell and trap data frames
cards_df = pd.concat([st_df, monster_df], ignore_index=True, axis=0).sort_values('Name').reset_index(drop=True)

# Create combined data frames with full data
full_df = pd.concat([cards_df, counter_df], ignore_index=True, axis=0).sort_values('Name').reset_index(drop=True)

print('Data merged')

## Save data

In [ ]:
full_df.to_csv(f'../data/all_cards_{timestamp.isoformat(timespec="minutes")}.csv', index = False)
print('Data saved')

# Check changes

## Load previous data

In [ ]:
# Get list of files
files_list = sorted(glob.glob('../data/all_cards_*.csv'), key=os.path.getctime, reverse=True)
# Get second newest file if exist
if len(files_list)>1:
    # Load csv avoiding converting "NA" to NaN
    previous_df = pd.read_csv(files_list[1], dtype=object, keep_default_na=False, na_values=[''])
    # Correct tuples
    previous_df['Primary type'] = previous_df['Secondary type'].dropna().apply(literal_eval)
    previous_df['Effect type'] = previous_df['Effect type'].dropna().apply(literal_eval)
    previous_df['Link Arrows'] = previous_df['Link Arrows'].dropna().apply(literal_eval)
    previous_df['Archseries'] = previous_df['Archseries'].dropna().apply(literal_eval)
    previous_df['Artwork'] = previous_df['Artwork'].dropna().apply(literal_eval)
    previous_df['Errata'] = previous_df['Errata'].dropna().apply(literal_eval)
    # Force dtypes to match current df
    previous_df = previous_df.astype(full_df[previous_df.columns].dtypes.to_dict())
    previous_ts = pd.to_datetime(os.path.basename(files_list[1]).split('_')[-1].rstrip('.csv'))
    print('File loaded')
else:
    previous_df = None
    print('No older files')

## Generate changelog

In [ ]:
if previous_df is None:
    print('Skipped')
else:
    changelog = generate_changelog(previous_df, full_df, col = 'Name')
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(f'../data/cards_changelog_{timestamp.isoformat(timespec="minutes")}_{previous_ts.isoformat(timespec="minutes")}.csv', index = True)
        print('Changelog saved')

# Data visualization

## Full data

In [ ]:
full_df

Full data available to download [here](../data)

## Card types

In [ ]:
cards_df.groupby('Card type').nunique()

In [ ]:
card_type_colors = [colors_dict[i] for i in full_df['Card type'].value_counts().index]
cards_df['Card type'].value_counts().plot.bar(figsize = (18,6), grid = True, rot=0, color = card_type_colors, title = 'Card type')
plt.show()

## Monsters

### Attributes

In [ ]:
print('Total number of attributes:', monster_df['Attribute'].nunique())

In [ ]:
monster_df.drop(columns=['Card type']).groupby('Attribute').nunique()

In [ ]:
attribute_colors = [colors_dict[i] for i in monster_df['Attribute'].value_counts().index]
monster_df['Attribute'].value_counts().plot.bar(figsize = (18,6), grid = True, rot=0, color = attribute_colors, title = 'Attribute')
plt.show()

### Primary types

In [ ]:
print('Total number of primary types:', monster_df['Primary type'].nunique())

In [ ]:
monster_df.drop(columns=['Card type']).groupby('Primary type').nunique()

#### Has effect discrimination

In [ ]:
has_effect = monster_df['Primary type'].where(monster_df['Effect type'].notna()).value_counts().rename('Effect')
no_effect = monster_df['Primary type'].where(monster_df['Effect type'].isna()).value_counts().rename('No Effect')
effect = pd.concat([has_effect,no_effect], axis=1).fillna(0).astype(int)
effect

In [ ]:
monster_type_colors = {'No Effect': colors_dict['Normal Monster'], 'Effect': [colors_dict[i] for i in effect.index]}
effect.plot.bar(figsize = (18,6), stacked = True, grid = True, rot=0,  legend=True, color = monster_type_colors, title = 'Primary types - Has effect')
# plt.yscale('log')
plt.show()

Obs: Normal monster can have effect if it is pendulum

#### Is pendulum discrimination

In [ ]:
not_pendulum = monster_df['Primary type'].where(monster_df['Pendulum Scale'].isna()).value_counts().rename('Not Pendulum')
is_pendulum = monster_df['Primary type'].where(monster_df['Pendulum Scale'].notna()).value_counts().rename('Pendulum')
pendulum = pd.concat([not_pendulum,is_pendulum], axis=1).fillna(0).astype(int)
pendulum

In [ ]:
monster_type_colors_b = {'Pendulum': colors_dict['Pendulum Monster'], 'Not Pendulum': [colors_dict[i] for i in pendulum.index]}
pendulum.plot.bar(figsize = (18,6), stacked = True, grid = True, rot=0, color = monster_type_colors_b, legend=True, title = 'Primary types - Is pendulum')
plt.show()

#### By attribute

In [ ]:
primmary_crosstab = pd.crosstab(cards_df['Primary type'],cards_df['Attribute'])
primmary_crosstab

In [ ]:
plt.figure(figsize = (16,10))
sns.heatmap(primmary_crosstab.T, annot=True, fmt="g", cmap='viridis', square=True, norm=mc.LogNorm())
plt.show()

### Secondary types

In [ ]:
exploded_secondary_type = monster_df.explode('Secondary type')
print('Total number of secondary types:', exploded_secondary_type['Secondary type'].nunique())

In [ ]:
exploded_secondary_type.drop(columns=['Card type', 'Link', 'Link Arrows']).groupby('Secondary type').nunique()

In [ ]:
secondary_type_colors = colors_dict['Effect Monster']
exploded_secondary_type['Secondary type'].value_counts().plot.bar(figsize = (18,6), stacked = True, grid = True, rot=0, color = secondary_type_colors, legend=False, title = 'Secondary type')
plt.show()

#### By attribute

In [ ]:
secondary_crosstab = pd.crosstab(exploded_secondary_type['Secondary type'],exploded_secondary_type['Attribute'])
secondary_crosstab

In [ ]:
plt.figure(figsize = (8,6))
sns.heatmap(secondary_crosstab[secondary_crosstab>0], annot=True, fmt="g", cmap='viridis', square=True)
plt.show()

#### By secondary type

In [ ]:
secondary_crosstab_b = pd.crosstab(exploded_secondary_type['Primary type'],exploded_secondary_type['Secondary type'], margins = True)
secondary_crosstab_b

In [ ]:
plt.figure(figsize = (10,7))
sns.heatmap(secondary_crosstab_b, annot=True, fmt="g", cmap='viridis', square=True, norm=mc.LogNorm())
plt.show()

### Monster types

In [ ]:
print('Total number of monster types:', monster_df['Monster type'].nunique())

In [ ]:
monster_df.drop(columns=['Card type']).groupby('Monster type').nunique()

In [ ]:
monster_type_colors = colors_dict['Monster Card']
monster_df['Monster type'].value_counts().plot.bar(figsize = (18,6), grid = True, rot=45, color = monster_type_colors, title = 'Monster type')
plt.show()

#### By Attribute

In [ ]:
monster_crosstab = pd.crosstab(cards_df['Monster type'],cards_df['Attribute'], dropna=False)
monster_crosstab

In [ ]:
plt.figure(figsize = (20,5))
sns.heatmap(monster_crosstab[monster_crosstab>0].T, annot=True, fmt="g", cmap='viridis', square=True, norm=mc.LogNorm())
plt.show()

#### By primary type

In [ ]:
monster_crosstab_b = pd.crosstab(cards_df['Monster type'],cards_df['Primary type'], dropna=False)
monster_crosstab_b

In [ ]:
plt.figure(figsize = (20,5))
sns.heatmap(monster_crosstab_b[monster_crosstab_b>0].T, annot=True, fmt="g", cmap='viridis', square=True, norm = mc.LogNorm())
plt.show()

#### By secondary type

In [ ]:
monster_crosstab_c = pd.crosstab(exploded_secondary_type['Monster type'],exploded_secondary_type['Secondary type'], dropna=False)
monster_crosstab_c

In [ ]:
plt.figure(figsize = (20,5))
sns.heatmap(monster_crosstab_c[monster_crosstab_c>0].T, annot=True, fmt="g", cmap='viridis', square=True, norm = mc.LogNorm())
plt.show()

### Effect type

In [ ]:
print('Total number of effect types:', monster_df['Effect type'].explode().nunique())

In [ ]:
monster_df[monster_df['Effect type'].notna()].drop(columns=['Card type']).explode('Effect type').groupby('Effect type').nunique()

In [ ]:
monster_effect_colors = colors_dict['Effect Monster']
monster_df['Effect type'].explode('Effect type').value_counts().plot.bar(figsize = (18,6), grid = True, color = monster_effect_colors, title='Effect type')
# plt.yscale('log')
plt.show()

### ATK

In [ ]:
print('Total number of ATK values:', monster_df['ATK'].nunique())

In [ ]:
monster_df.drop(columns=['Card type']).groupby('ATK').nunique().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce'))

In [ ]:
atk_colors = colors_dict['Monster Card']
monster_df['DEF'].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce')).plot.bar(figsize = (18,6), grid = True, color = atk_colors, title = 'ATK')
plt.show()

### DEF

In [ ]:
print('Total number of DEF values:', monster_df['DEF'].nunique())

In [ ]:
monster_df.drop(columns=['Card type']).groupby('DEF').nunique().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce'))

In [ ]:
def_colors = colors_dict['Monster Card']
monster_df['DEF'].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce')).plot.bar(figsize = (18,6), grid = True, color = def_colors, title = 'DEF')
plt.show()

### Level/Rank

In [ ]:
monster_df.drop(columns=['Card type', 'Link', 'Link Arrows']).groupby('Level/Rank').nunique().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce'))

In [ ]:
stars_colors = colors_dict['Level']
monster_df['Level/Rank'].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce')).plot.bar(figsize = (18,6), grid = True, rot=0, color= stars_colors, title = 'Level/Rank')
plt.show()

#### ATK statistics

In [ ]:
monster_df[['Level/Rank','ATK']].apply(pd.to_numeric, errors = 'coerce').dropna().astype(int).groupby('Level/Rank').describe()

#### DEF statistics

In [ ]:
monster_df[['Level/Rank','DEF']].apply(pd.to_numeric, errors = 'coerce').dropna().astype(int).groupby('Level/Rank').describe()

### Pendulum scale

In [ ]:
monster_df.drop(columns=['Card type', 'Link', 'Link Arrows']).groupby('Pendulum Scale').nunique().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce'))

In [ ]:
scales_colors = colors_dict['Pendulum Monster']
monster_df['Pendulum Scale'].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce')).plot.bar(figsize = (18,6), grid = True, rot=0, color = scales_colors, title = 'Pendulum scale')
plt.show()

#### ATK statistics

In [ ]:
monster_df[['Pendulum Scale','ATK']].apply(pd.to_numeric, errors = 'coerce').dropna().astype(int).groupby('Pendulum Scale').describe()

#### DEF statistics

In [ ]:
monster_df[['Pendulum Scale','DEF']].apply(pd.to_numeric, errors = 'coerce').dropna().astype(int).groupby('Pendulum Scale').describe()

#### Level/Rank statistics

In [ ]:
monster_df[['Pendulum Scale','Level/Rank']].apply(pd.to_numeric, errors = 'coerce').dropna().astype(int).groupby('Pendulum Scale').describe()

### Link

In [ ]:
monster_df.drop(columns=['Card type', 'Primary type', 'Secondary type','Level/Rank','DEF','Pendulum Scale']).groupby('Link').nunique().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce'))

In [ ]:
link_colors = colors_dict['Link Monster']
monster_df['Link'].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce')).plot.bar(figsize = (18,6), grid = True, rot=0, color = link_colors, title='Link')
plt.show()

#### ATK statistics

In [ ]:
monster_df[['Link','ATK']].apply(pd.to_numeric, errors = 'coerce').dropna().astype(int).groupby('Link').describe()

### Link Arrows

#### By combination

In [ ]:
print('Total number of link arrow combinations:', monster_df['Link Arrows'].nunique())

In [ ]:
monster_df.drop(columns=['Card type', 'Primary type', 'Level/Rank', 'Pendulum Scale', 'Link', 'Secondary type', 'DEF']).groupby('Link Arrows').nunique()

In [ ]:
arrows_colors = colors_dict['Link Monster']
monster_df['Link Arrows'].value_counts().plot.barh(figsize = (10,20), grid = True, color = arrows_colors, title = 'Link arrows combinations')
plt.show()

#### By unique

In [ ]:
monster_df[monster_df['Link Arrows'].notna()].drop(columns=['Card type', 'Primary type', 'Level/Rank', 'Pendulum Scale', 'Secondary type', 'DEF']).explode('Link Arrows').groupby('Link Arrows').nunique()

In [ ]:
arrows_colors_b = colors_dict['Link Monster']
monster_df['Link Arrows'].explode('Link Arrows').value_counts().plot.bar(figsize = (18,6), grid = True, color = arrows_colors_b, title = 'Link arrows')
plt.show()

#### By link

In [ ]:
arrow_per_link = monster_df[['Link Arrows','Link']].explode('Link Arrows').dropna()
arrow_crosstab = pd.crosstab(arrow_per_link['Link Arrows'],arrow_per_link['Link'])
arrow_crosstab

In [ ]:
plt.figure(figsize = (10,6))
sns.heatmap(arrow_crosstab[arrow_crosstab>0].T, annot=True, fmt="g", cmap='viridis', square=True, norm=mc.LogNorm())
plt.show()

## Spell & Trap

### Properties

In [ ]:
print('Total number of properties:', st_df['Property'].nunique())

In [ ]:
st_df.drop(columns=['Card type']).groupby('Property').nunique()

In [ ]:
st_colors = [colors_dict[i] for i in cards_df[['Card type','Property']].value_counts().index.get_level_values(0)]
st_df['Property'].value_counts().plot.bar(figsize = (18,6), grid = True, rot=45, color = st_colors, title = 'Property')
plt.show()

### Effect type

In [ ]:
print('Total number of effect types:', st_df['Effect type'].explode().nunique())

In [ ]:
st_df.explode('Effect type').groupby('Effect type').nunique()

#### Spell & Trap discrimination

In [ ]:
spell = spell_df['Effect type'].explode('Effect type').value_counts().rename('Spell Card')
trap = trap_df['Effect type'].explode('Effect type').value_counts().rename('Trap Card')
st_diff = pd.concat([spell, trap], axis = 1).fillna(0).astype(int)
st_diff

In [ ]:
st_diff_colors = {'Spell Card': colors_dict['Spell Card'], 'Trap Card': colors_dict['Trap Card']}
st_diff.plot.bar(figsize = (18,6), stacked = True, grid = True, rot=45, color = st_diff_colors, title = 'Effect type')
plt.show()

## Archseries

In [ ]:
exploded_archseries = cards_df.explode('Archseries')
print('Total number of Archseries:', exploded_archseries['Archseries'].nunique())

In [ ]:
exploded_archseries.groupby('Archseries').nunique()

In [ ]:
exploded_archseries['Archseries'].value_counts().plot.barh(figsize = (10,200), grid = True, title = 'Archtypes/Series')
plt.show()

### By card type

In [ ]:
archseries_crosstab = pd.crosstab(exploded_archseries['Archseries'],exploded_archseries['Card type'], margins = True)
archseries_crosstab

### By primary type

In [ ]:
archseries_crosstab_b = pd.crosstab(exploded_archseries['Archseries'],exploded_archseries['Primary type'], margins = True)
archseries_crosstab_b



### By secondary type

In [ ]:
exploded_archseries_secondary_type = exploded_archseries[['Archseries', 'Secondary type']].explode('Secondary type')
archseries_crosstab_c = pd.crosstab(exploded_archseries_secondary_type['Archseries'],exploded_archseries_secondary_type['Secondary type'], margins = True)
archseries_crosstab_c

### By monster type

In [ ]:
archseries_crosstab_d = pd.crosstab(exploded_archseries['Archseries'],exploded_archseries['Monster type'], margins = True)
archseries_crosstab_d

### By property

In [ ]:
archseries_crosstab_e = pd.crosstab(exploded_archseries['Archseries'],exploded_archseries['Property'], margins = True)
archseries_crosstab_e

## Artworks

In [ ]:
print('Total number of cards with edited or alternate artworks:', cards_df['Artwork'].count())

In [ ]:
cards_df[['Name','Password','TCG status','OCG status','Artwork']][cards_df['Artwork'].notna()]

In [ ]:
artwork_value_counts = cards_df['Artwork'].value_counts()
plt.figure(figsize=(20,8))
plt.title('Artworks')
venn2(subsets = (artwork_value_counts[('Alternate',)], artwork_value_counts[('Edited',)],artwork_value_counts[('Alternate','Edited')]), set_labels = ('Alternate artwork', 'Edited artwork'))
plt.show()

### By card type

In [ ]:
artwork_crosstab = pd.crosstab(cards_df['Artwork'], cards_df['Card type'])
artwork_crosstab

### By primary type

In [ ]:
artwork_crosstab_b = pd.crosstab(cards_df['Artwork'], cards_df['Primary type'])
artwork_crosstab_b

More granularity is unnecessary

## Errata

In [ ]:
print('Total number of cards with name or type errata:', cards_df['Errata'].count())

In [ ]:
cards_df[['Name','Password','TCG status','OCG status','Errata']][cards_df['Errata'].notna()]

In [ ]:
errata_value_counts = cards_df['Errata'].value_counts()
plt.figure(figsize=(20,8))
plt.title('Errata')
venn2(subsets = (errata_value_counts[('Name',)], errata_value_counts[('Type',)],errata_value_counts[('Name','Type')]), set_labels = ('Name Errata', 'Type errata'))
plt.show()

### By card type

In [ ]:
errata_crosstab = pd.crosstab(cards_df['Errata'], cards_df['Card type'])
errata_crosstab

### By primary type

In [ ]:
errata_crosstab_b = pd.crosstab(cards_df['Errata'], cards_df['Primary type'])
errata_crosstab_b

More granularity is unnecessary

### By artwork

In [ ]:
errata_crosstab_c = pd.crosstab(cards_df['Artwork'], cards_df['Errata'])
errata_crosstab_c

## TCG & OCG status

### TGC status

In [ ]:
print('Total number of TCG status:', cards_df['TCG status'].nunique())

In [ ]:
cards_df.groupby('TCG status', dropna=False).nunique()

In [ ]:
cards_df['TCG status'].value_counts(dropna = False).plot.bar(figsize = (18,6), logy=True, grid = True, rot=45, title = 'TCG status')
plt.show()

#### By card type

In [ ]:
# Remove unlimited
tcg_crosstab = pd.crosstab(cards_df['Card type'], cards_df['TCG status']).drop(['Unlimited'], axis=1)
tcg_crosstab

In [ ]:
plt.figure(figsize = (12,6))
sns.heatmap(tcg_crosstab[tcg_crosstab>0], annot=True, fmt="g", cmap='viridis', norm=mc.LogNorm())
plt.show()

#### By monster type

In [ ]:
# Remove unlimited
tcg_crosstab_b = pd.crosstab(cards_df['Monster type'], cards_df['TCG status']).drop(['Unlimited'], axis=1)
tcg_crosstab_b

In [ ]:
plt.figure(figsize = (20,5))
sns.heatmap(tcg_crosstab_b[tcg_crosstab_b>0].T, annot=True, fmt="g", cmap='viridis', square = True)
plt.show()

#### By archseries

In [ ]:
# Remove unlimited
tcg_crosstab_c = pd.crosstab(exploded_archseries['Archseries'].where(exploded_archseries['OCG status']!='Unlimited'), exploded_archseries['TCG status'], margins = True)
tcg_crosstab_c

### OCG status

In [ ]:
print('Total number of OCG status:', cards_df['OCG status'].nunique())

In [ ]:
cards_df.groupby('OCG status', dropna=False).nunique()

In [ ]:
cards_df['OCG status'].value_counts(dropna = False).plot.bar(figsize = (18,6), logy=True, grid = True, rot=45, title = 'TCG status')
plt.show()

#### By card type

In [ ]:
# Remove unlimited
ocg_crosstab = pd.crosstab(cards_df['Card type'], cards_df['OCG status']).drop(['Unlimited'], axis=1)
ocg_crosstab

In [ ]:
plt.figure(figsize = (12,6))
sns.heatmap(ocg_crosstab[ocg_crosstab>0], annot=True, fmt="g", cmap='viridis')
plt.show()

#### By monster type

In [ ]:
ocg_crosstab_b = pd.crosstab(cards_df['Monster type'], cards_df['OCG status']).drop(['Unlimited'], axis=1)
ocg_crosstab_b

In [ ]:
plt.figure(figsize = (20,5))
sns.heatmap(ocg_crosstab_b[ocg_crosstab_b>0].T, annot=True, fmt="g", cmap='viridis', square = True)
plt.show()

#### By archseries

In [ ]:
# Remove unlimited
ocg_crosstab_c = pd.crosstab(exploded_archseries['Archseries'].where(exploded_archseries['OCG status']!='Unlimited'), exploded_archseries['OCG status'], margins = True)
ocg_crosstab_c

### TCG vs. OCG status

In [ ]:
cg_crosstab = pd.crosstab(cards_df['OCG status'],cards_df['TCG status'], dropna=False, margins = False)
cg_crosstab

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(cg_crosstab[cg_crosstab>0], annot=True, fmt="g", cmap='viridis', square=True, norm=mc.LogNorm())
plt.show()

# Extras

## Multiple secondary types

In [ ]:
cards_df.dropna(subset='Secondary type', axis=0)[[len(x)>1 for x in cards_df['Secondary type'].dropna()]]

## Not yet released

In [ ]:
cards_df.loc[cards_df['OCG status'] == 'Not yet released'].loc[full_df['TCG status'] == 'Not yet released']

## Counters and Tokens

In [ ]:
counter_df.dropna(how='all',axis=1)

# Epilogue

In [ ]:
footer()

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
! jupyter nbconvert Cards.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

## Git

In [ ]:
! git add "../*[Cc]ards*"

In [ ]:
! git commit -m {"'Cards update-" + timestamp.isoformat() + "'"}